In [1]:
import zipfile
import os
import numpy as np
import pandas as pd
from sklearn.metrics import recall_score, f1_score, matthews_corrcoef
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import time

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns



2025-06-22 19:10:29.769948: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-22 19:10:29.769995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-22 19:10:29.773383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-22 19:10:29.787149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-22 19:10:30.947880: W tensorflow/compiler/tf2

In [2]:
import tensorflow as tf

# Verificar si detecta GPUs
print("GPUs disponibles:", tf.config.list_physical_devices('GPU'))


GPUs disponibles: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-06-22 19:10:32.394540: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-22 19:10:32.445446: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-22 19:10:32.445757: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
base_dir = './Banana Ripeness Classification Dataset V2'
print(os.listdir(base_dir))

['train', 'valid', 'test']


In [4]:
lr ="001"
Et = "10"
l_rate = 0.001
Epoch_test = 10

In [5]:
def save_test_accuracy(experiment_name, test_accuracy, filename='test_accuracy_log.csv'):
    new_entry = pd.DataFrame([{
        'experiment': experiment_name,
        'test_accuracy': test_accuracy
    }])

    if os.path.exists(filename):
        df_existing = pd.read_csv(filename)
        # Eliminar cualquier fila con el mismo experimento
        df_existing = df_existing[df_existing['experiment'] != experiment_name]
        # Agregar nueva entrada
        df_combined = pd.concat([df_existing, new_entry], ignore_index=True)
    else:
        df_combined = new_entry

    df_combined.to_csv(filename, index=False)
    print(f"Test accuracy for '{experiment_name}' saved to '{filename}'.")

In [6]:
class MetricsLogger(Callback):
    def __init__(self, validation_data, experiment_name=None):
        super(MetricsLogger, self).__init__()
        self.validation_data = validation_data
        self.history = []
        self.experiment_name = experiment_name
        self.epoch_start_time = None

    def on_epoch_begin(self, epoch, logs=None):
        self.epoch_start_time = time.time()

    def on_epoch_end(self, epoch, logs=None):
        epoch_duration = time.time() - self.epoch_start_time

        val_data, val_labels = self.validation_data
        val_preds = self.model.predict(val_data, verbose=0)
        val_preds_classes = np.argmax(val_preds, axis=1)
        val_true_classes = np.argmax(val_labels, axis=1)

        recall = recall_score(val_true_classes, val_preds_classes, average='macro')
        f1 = f1_score(val_true_classes, val_preds_classes, average='macro')
        mcc = matthews_corrcoef(val_true_classes, val_preds_classes)

        logs = logs or {}

        # Entrenamiento
        train_accuracy = logs.get('accuracy')
        train_loss = logs.get('loss')

        # Validación
        val_accuracy = logs.get('val_accuracy')
        val_loss = logs.get('val_loss')

        self.history.append({
            'experiment': self.experiment_name if self.experiment_name else 'default',
            'epoch': epoch + 1,
            'accuracy': train_accuracy,
            'loss': train_loss,
            'val_accuracy': val_accuracy,
            'val_loss': val_loss,
            'recall': recall,
            'f1_score': f1,
            'mcc': mcc,
            'epoch_time': round(epoch_duration, 2)
        })

    def save_to_csv(self, filename='metrics_log.csv'):
        new_df = pd.DataFrame(self.history)

        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)

            # Eliminar filas con experimentos ya existentes
            existing_df = existing_df[~existing_df['experiment'].isin(new_df['experiment'])]

            # Combinar con los nuevos datos
            combined_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            combined_df = new_df

        combined_df.to_csv(filename, index=False)
        print(f"Metrics saved to '{filename}' with {len(new_df)} new rows.")


In [7]:
image_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

valid_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,)

train_generator = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'valid'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
)

test_generator = valid_test_datagen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Importante para la matriz de confusión
)


Found 9440 images belonging to 4 classes.
Found 2023 images belonging to 4 classes.
Found 2015 images belonging to 4 classes.


In [8]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

# Igual que antes
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)  # 4 clases como antes

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate= l_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

2025-06-22 19:10:33.028290: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-22 19:10:33.028488: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-06-22 19:10:33.028577: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [9]:
def generator_to_arrays(generator):
    X_list, y_list = [], []
    for i in range(len(generator)):
        x, y = generator[i]
        X_list.append(x)  # <- aquí
        y_list.append(y)
    return np.concatenate(X_list), np.concatenate(y_list)

X_val, y_val = generator_to_arrays(valid_generator)
# Crear callback personalizado
metrics_logger = MetricsLogger(validation_data=(X_val, y_val), experiment_name=f'resnet50_{l_rate}_{Epoch_test}')


In [ ]:
with tf.device('/GPU:0'):
    history = model.fit(
        train_generator,
        epochs=Epoch_test,
        validation_data=valid_generator,
        callbacks=[metrics_logger]
    )

# Guardar métricas en CSV (
metrics_logger.save_to_csv('resultados_modelos.csv')

Epoch 1/10


2025-06-22 19:10:48.607063: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-06-22 19:10:51.008083: I external/local_xla/xla/service/service.cc:168] XLA service 0x7faa111ad360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-22 19:10:51.008119: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A4000, Compute Capability 8.6
2025-06-22 19:10:51.015659: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750619451.113036    8901 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


295/295 [==============================] - 98s 316ms/step - loss: 0.3087 - accuracy: 0.8910 - val_loss: 0.1314 - val_accuracy: 0.9471
Epoch 2/10
295/295 [==============================] - 104s 346ms/step - loss: 0.1814 - accuracy: 0.9344 - val_loss: 0.1455 - val_accuracy: 0.9511
Epoch 3/10
295/295 [==============================] - 104s 352ms/step - loss: 0.1667 - accuracy: 0.9405 - val_loss: 0.0973 - val_accuracy: 0.9654
Epoch 4/10
295/295 [==============================] - 102s 345ms/step - loss: 0.1456 - accuracy: 0.9488 - val_loss: 0.1036 - val_accuracy: 0.9595
Epoch 5/10
295/295 [==============================] - 99s 335ms/step - loss: 0.1374 - accuracy: 0.9517 - val_loss: 0.0947 - val_accuracy: 0.9649
Epoch 6/10
295/295 [==============================] - 102s 345ms/step - loss: 0.1355 - accuracy: 0.9493 - val_loss: 0.0901 - val_accuracy: 0.9664
Epoch 7/10
295/295 [==============================] - 103s 349ms/step - loss: 0.1301 - accuracy: 0.9496 - val_loss: 0.1133 - val_accuracy

In [ ]:
# Predicciones
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

# Reporte y matriz de confusión
print('Classification Report')
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))

print('Confusion Matrix')
cm = confusion_matrix(test_generator.classes, y_pred)

# Visualizar la matriz
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=test_generator.class_indices.keys(),
            yticklabels=test_generator.class_indices.keys(), cmap='Blues')
plt.title(f"Resultados modelo ResNet50 learning_rate={lr} epoch={Et} ")
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig(f'Matrices de confusion/RESNET50/matriz_confusion_lr{lr}_epoch{Et}.png', bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score

test_accuracy = accuracy_score(test_generator.classes, y_pred)
experiment_name = f'resnet50_{l_rate}_{Epoch_test}'

save_test_accuracy(experiment_name, test_accuracy)

In [ ]:
model.save(f'modelos/RESNET50_{lr}_{Et}.h5')